In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
)
from theme import apply_theme
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [ ]:
data_release='2021-04-27'

df = pd.read_csv(join("..", "data", "Phase2.1SurvivalRSummariesPublic", "ToShare", "table.stay.rmDead.toShare.csv"))

df = df.drop(columns=['Unnamed: 0'])
#df = df.rename(columns={"n.early": 'First', 'n.late': "Second", 'week.setting': 'week'})

colors = ['#E79F00', '#0072B2', '#D45E00', '#CB7AA7', '#029F73', '#57B4E9']

sites = ['META', 'APHP', 'FRBDX', 'ICSM', 'UKFR', 'NWU', 'BIDMC', 'MGB', 'UCLA', 'UMICH', 'UPENN', 'UPITT', 'VA1', 'VA2', 'VA3', 'VA4', 'VA5']
site_colors = ['black', '#0072B2', '#0072B2', '#0072B2', '#0072B2', '#CB7AA7', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00','#D45E00','#D45E00']

#df = pd.melt(df, id_vars=['siteid', 'week'], value_vars=['First', 'Second'], var_name='wave', value_name='n')

df.siteid = df.siteid.apply(lambda x: x.upper())
df.week = df.week.apply(lambda x: 'Week3' if x == 'week3' else x.capitalize())
df['week'] = df['week'].apply(lambda x: {
    'Week1': '<=1 Week',
    'Week2': '1=2 Week',
    'Week3': '2+ Week'
}[x])

siteid = df.siteid.unique().tolist()
print(df.siteid.unique().tolist())

df

# All Sites

In [ ]:
point=alt.OverlayMarkDef(filled=False, fill='white', strokeWidth=2)

def plot_stay(df=None, wave='First'):
    d = df.copy()
    
    d = d[d.siteid != 'META']
    d = d[d.wave == wave]
        
    plot = alt.Chart(
        d
    ).mark_bar(
        size=30,
        strokeWidth=0,
        stroke='black'
    ).encode(
        x=alt.X("week:N", title=None, axis=alt.Axis(labelAngle=0, tickCount=10, labels=False if wave=='early' else True), scale=alt.Scale(padding=0), sort=['Week1', 'Week2', 'Week3']),
        y=alt.Y("n:Q", title=None, scale=alt.Scale(zero=True, domain=[0, 1]), axis=alt.Axis(format='%')),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['First', 'Second'], range=['#D45E00', '#0072B2']), title="Wave")
    ).properties(
        width=150,
        height=100
    )

    text = plot.mark_text(
        size=16, dx=0, dy=-5, color='white', baseline='bottom', fontWeight=500
    ).encode(
    #     x=alt.X('month:N'),
    #     y=alt.Y('value:Q', stack='zero'),
        x=alt.X("week:N", title=None, axis=alt.Axis(labels=False)),
        y=alt.Y("n:Q", title=None),
    #     detail='cat:N',
        text=alt.Text('n:Q', format='.0%'),
    #     order="order:O",
    #     opacity=alt.Opacity('visibility:N', scale=alt.Scale(domain=[True, False], range=[1, 0]))
    )
        
    plot = (plot + text).facet(
        column=alt.Column("siteid:N", header=alt.Header(title=None, labels=True if wave=='First' else False), sort=sites)
    ).resolve_scale(color='shared')

#     plot = plot.properties(
#         title={
#             "text": [
#                 f"Length Of Stay"
#             ],
#             "dx": 30,
#             "subtitle": [
#                 get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#             ],
#             "subtitleColor": "gray",
#         }
#     )

    return plot


e = plot_stay(df=df, wave='First')
l = plot_stay(df=df, wave='Second')

print(df)

plot = alt.vconcat(e, l, spacing=10).properties(
    title={
        "text": [
            f"Length Of Stay"
        ],
        "dx": 40,
#         "subtitle": [
#             get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#         ],
        "subtitleColor": "gray",
    }
)

plot = apply_theme(
    plot,
    axis_y_title_font_size=16,
    title_anchor='start',
    header_label_orient='top',
    legend_orient='top',
    legend_title_orient='left',
    axis_label_font_size=14,
    header_label_font_size=16,
    point_size=100
)

plot
save(plot,join("..", "result", "final-stay-site-rmDead.png"), scalefactor=8.0)



In [ ]:
point=alt.OverlayMarkDef(filled=False, fill='white', strokeWidth=2)

def plot_stay(df=None, wave='early'):
    d = df.copy()
    
    d = d[d.siteid == 'META']
#     d = d[d.wave == wave]
        
    plot = alt.Chart(
        d
    ).mark_bar(
        size=30,
        strokeWidth=0,
        stroke='black'
    ).encode(
        x=alt.X("wave:N", title=None, axis=alt.Axis(labelAngle=0, tickCount=10, labels=True), scale=alt.Scale(padding=0), sort=['Week1', 'Week2', 'Week3']),
        y=alt.Y("n:Q", title=None, scale=alt.Scale(zero=False, domain=[0, 0.65]), axis=alt.Axis(format="%")),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['First', 'Second'], range=['#D45E00', '#0072B2']), title="Wave")
    ).properties(
        width=100,
        height=180
    )
    tick = plot.mark_errorbar(
        opacity=0.7 #, color='black'
    ).encode(
        x=alt.X("wave:N"),
        y=alt.Y("ci_l:Q"),
        y2=alt.Y2("ci_u:Q"),
        stroke=alt.value('black'),
        strokeWidth=alt.value(1)
    )
      
    text = plot.mark_text(
        size=16, dx=0, dy=-5, color='white', baseline='bottom', fontWeight=500
    ).encode(
    #     x=alt.X('month:N'),
    #     y=alt.Y('value:Q', stack='zero'),
        x=alt.X("wave:N", title=None, axis=alt.Axis(labels=False)),
        y=alt.Y("n:Q", title=None),
    #     detail='cat:N',
        text=alt.Text('n:Q', format='.0%'),
    #     order="order:O",
    #     opacity=alt.Opacity('visibility:N', scale=alt.Scale(domain=[True, False], range=[1, 0]))
    )
    
    plot = (plot + text+tick).facet(
        column=alt.Column("week:N", header=alt.Header(title=None, labels=True), sort=sites)
    ).resolve_scale(color='shared')

#     plot = plot.properties(
#         title={
#             "text": [
#                 f"Length Of Stay"
#             ],
#             "dx": 30,
#             "subtitle": [
#                 get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#             ],
#             "subtitleColor": "gray",
#         }
#     )

    return plot

e = plot_stay(df=df, wave='early')
# l = plot_stay(df=df, wave='late')

# plot = alt.hconcat(e, l, spacing=30).resolve_scale(y='shared').properties(
plot = e.properties(
    title={
        "text": [
            f"Meta-Analysis Of Length Of Stay"
        ],
        "dx": 30,
#         "subtitle": [
#             get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#         ],
        "subtitleColor": "gray",
    }
)

plot = apply_theme(
    plot,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='bottom',
    legend_title_orient='left',
    axis_label_font_size=14,
    header_label_orient='top',
    header_label_font_size=16,
    point_size=100
)

plot
save(plot,join("..", "result", "final-stay-meta-rmDead.png"), scalefactor=8.0)



## Country

In [ ]:
print(df)
point=alt.OverlayMarkDef(filled=False, fill='white', strokeWidth=2)
#df['ci_l'] = df.n * 0.9
#df['ci_u'] = df.n * 1.1

def plot_stay(df=None, wave='early', country=None):
    d = df.copy()
    
    d = d[d.siteid == country]
#     d = d[d.wave == wave]
        
    plot = alt.Chart(
        d
    ).mark_bar(
        size=30,
        strokeWidth=0,
        stroke='black'
    ).encode(
        x=alt.X("wave:N", title=None, axis=alt.Axis(labelAngle=0, tickCount=10, labels=True), scale=alt.Scale(padding=0), sort=['Week1', 'Week2', 'Week3']),
        y=alt.Y("n:Q", title=None, scale=alt.Scale(zero=True, domain=[0, 0.9]), axis=alt.Axis(format="%")),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['First', 'Second'], range=['#D45E00', '#0072B2']), title="Wave")
    ).properties(
        width=100,
        height=180
    )
    
    tick = plot.mark_errorbar(
        opacity=0.7 #, color='black'
    ).encode(
        x=alt.X("wave:N"),
        y=alt.Y("ci_l:Q"),
        y2=alt.Y2("ci_u:Q"),
        stroke=alt.value('black'),
        strokeWidth=alt.value(1)
    )
        

    text = plot.mark_text(
        size=16, dx=0, dy=-5, color='white', baseline='bottom', fontWeight=500
    ).encode(
    #     x=alt.X('month:N'),
    #     y=alt.Y('value:Q', stack='zero'),
        x=alt.X("wave:N", title=None, axis=alt.Axis(labels=False)),
        y=alt.Y("n:Q", title=None),
    #     detail='cat:N',
        text=alt.Text('n:Q', format='.0%'),
    #     order="order:O",
    #     opacity=alt.Opacity('visibility:N', scale=alt.Scale(domain=[True, False], range=[1, 0]))
    )
    
    plot = (plot + text+tick).facet(
        column=alt.Column("week:N", header=alt.Header(title=None, labels=True), sort=sites)
    ).resolve_scale(color='shared')

    plot = plot.properties(
    title={
        "text": [
            country.replace("META-", "")
        ],
        "dx": 30,
#         "subtitle": [
#             get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#         ],
        "subtitleColor": "gray",
    }
)
#     plot = plot.properties(
#         title={
#             "text": [
#                 f"Length Of Stay"
#             ],
#             "dx": 30,
#             "subtitle": [
#                 get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#             ],
#             "subtitleColor": "gray",
#         }
#     )

    return plot

country_list1=["META-USA", "META-FRANCE"]
country_list2=["META-ITALY", "META-GERMANY"]



e1 = alt.hconcat(*(plot_stay(df=df, wave='early', country=country) for country in country_list1))
e2 = alt.hconcat(*(plot_stay(df=df, wave='early', country=country) for country in country_list2))

e=alt.vconcat(e1,e2)
# l = plot_stay(df=df, wave='late')

# plot = alt.hconcat(e, l, spacing=30).resolve_scale(y='shared').properties(


plot = apply_theme(
    e,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='bottom',
    legend_title_orient='left',
    axis_label_font_size=14,
    header_label_orient='top',
    header_label_font_size=16,
    point_size=100
)

plot
save(plot,join("..", "result", "final-stay-country-rmDead.png"), scalefactor=8.0)

